# FCIM.M.IA - Artificial Intelligence

> **Lab 3:** Chess Engine \\
> **Performed by:** Trifan Denis, group TI-231M \\
> **Verified by:** Mihail Gavrilita, asist. univ.

## Task 1 -- Implement the Minimax algorithm with the following scoring function: Score = MaterialValue + PositionalValue. For computing the MaterialValue, each piece is assigned a value (e.g., Pawn = 1, Knight = 3, Bishop = 3, Rook = 5, Queen = 9). Then you sum these values for your pieces and subtract the value of the pieces of the opponent. For computing the PositionalValue, you should take into account the position of each pieces on the board (e.g the more squares a pawn has travelled, the higher their PositionalValue etc.). You should then subtract the opponent’s PositionalValue from your pieces’ PositionalValue.

In [1]:
def minimax(board, depth, max_player, save_move, data):
    # check if depth is 0 and evaluate the board for score
    if depth == 0 or board.is_terminal():
        data[1] = board.evaluate()
        return data

    # recursively check if it's maximizing move
    if max_player:
        # initialize max score with -inf
        max_eval = -math.inf
        for i in range(8):
            for j in range(8):
                # check if element on position i,j is a chess piece and if piece belong to oponent color
                if isinstance(board[i][j], ChessPiece) and board[i][j].color != board.get_player_color():
                    piece = board[i][j]
                    # filter valid moves for the piece
                    moves = piece.filter_moves(piece.get_moves(board), board)
                    # iterate for every move
                    for move in moves:
                        # make the move
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        # call oponent's move
                        evaluation = minimax(board, depth - 1, False, False, data)[1]
                        if save_move:
                            # update max value move
                            if evaluation >= max_eval:
                                if evaluation > data[1]:
                                    data.clear()
                                    data[1] = evaluation
                                    data[0] = [piece, move, evaluation]
                                elif evaluation == data[1]:
                                    data[0].append([piece, move, evaluation])
                        # undo last move
                        board.unmake_move(piece)
                        # update max move
                        max_eval = max(max_eval, evaluation)
        return data
    else:
        # for oponent minimizing move
        min_eval = math.inf
        for i in range(8):
            for j in range(8):
                # check if element on position i,j is a chess piece and if piece belong to player itself
                if isinstance(board[i][j], ChessPiece) and board[i][j].color == board.get_player_color():
                    piece = board[i][j]
                    # filter moves
                    moves = piece.get_moves(board)
                    for move in moves:
                        # make the move of piece
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        # make oponent move
                        evaluation = minimax(board, depth - 1, True, False, data)[1]
                        board.unmake_move(piece)
                        # update min value
                        min_eval = min(min_eval, evaluation)
        return data

def evaluate(self):
    white_points = 0
    black_points = 0
    for i in range(8):
        for j in range(8):
            if isinstance(self[i][j], ChessPiece):
                piece = self[i][j]
                if piece.color == 'white':
                    white_points += piece.get_score()
                    white_points += piece.x + piece.y
                else:
                    black_points += piece.get_score()
                    black_points += piece.x + piece.y
    if self.game_mode == 0:
        return black_points - white_points
    return white_points - black_points

## Task 2 -- Implement Alpha-Beta Prunning.

In [2]:
def minimax_alpha_beta_pruning(board, depth, alpha, beta, max_player, save_move, data):
    if depth == 0 or board.is_terminal():
        data[1] = board.evaluate()
        return data

    if max_player:
        max_eval = -math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color != board.get_player_color():
                    piece = board[i][j]
                    moves = piece.filter_moves(piece.get_moves(board), board)
                    for move in moves:
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        evaluation = minimax_alpha_beta_pruning(board, depth - 1, alpha, beta, False, False, data)[1]
                        if save_move:
                            if evaluation >= max_eval:
                                if evaluation > data[1]:
                                    data.clear()
                                    data[1] = evaluation
                                    data[0] = [piece, move, evaluation]
                                elif evaluation == data[1]:
                                    data[0].append([piece, move, evaluation])
                        board.unmake_move(piece)
                        max_eval = max(max_eval, evaluation)
                        # update alpha value to max evaluation
                        alpha = max(alpha, evaluation)
                        # terminate move
                        if beta <= alpha:
                            break
        return data
    else:
        min_eval = math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color == board.get_player_color():
                    piece = board[i][j]
                    moves = piece.get_moves(board)
                    for move in moves:
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        evaluation = minimax_alpha_beta_pruning(board, depth - 1, alpha, beta, True, False, data)[1]
                        board.unmake_move(piece)
                        min_eval = min(min_eval, evaluation)
                        # update beta with min evaluation
                        beta = min(beta, evaluation)
                        if beta <= alpha:
                            break
        return data

## Task 3 -- Implement an improved scoring (evaluation) method for Minimax. For example, you could add values like KingSafetyValue, MobilityValue (nr. of legal moves to each side), PawnStructureValue (can include penalties for isolated pawns, doubled pawns, and bonuses for passed pawns or a strong pawn chain), etc. You can also use heuristic evaluation functions. Be creative! 

In [3]:
def calculate_king_safety(self, king):
    safety_value = 0
    king_x, king_y = king.x, king.y
    player_color = king.color

    # evaluate pawn shield in front of the king
    pawn_shield = self.get_pawn_shield(king_x, king_y, player_color)
    safety_value += len(pawn_shield)

    # evaluate open files near the king
    open_files = self.get_open_files(king_x, king_y)
    safety_value += len(open_files) * 1

    # evaluate piece placement near the king (knights, bishops)
    piece_placement_value = self.evaluate_piece_placement(king_x, king_y, player_color)
    safety_value += piece_placement_value

    return safety_value

def get_pawn_shield(self, king_x, king_y, color):
    shield = []
    king_row, king_col = king_x, king_y
    # check coordinates of pawns in front of the king - left-forward, forward, rigth-forward
    pawn_offsets = [(1, -1), (1, 0), (1, 1)] if color == 'White' else [(-1, -1), (-1, 0), (-1, 1)]
    for offset in pawn_offsets:
        row, col = king_row + offset[0], king_col + offset[1]
        if self.is_valid_move(row, col) and isinstance(self[row][col], Pawn) and self[row][col].color == color:
            shield.append((row, col))
    return shield

def get_open_files(self, king_x, king_y):
    files = set()
    king_row, king_col = king_x, king_y
    for col in range(8):
        if col != king_col:
            open_file = True
            for row in range(8):
                if self[row][col] is not None:
                    open_file = False
                    break
            if open_file:
                files.add(col)
    return files

def evaluate_piece_placement(self, king_x, king_y, color):
    placement_value = 0
    king_row, king_col = king_x, king_y
    for i in range(-1, 2):
        for j in range(-1, 2):
            row, col = king_row + i, king_col + j
            if self.is_valid_move(row, col) and isinstance(self[row][col], (Knight, Bishop)):
                placement_value += 0.5 if self[row][col].color == color else -0.5
    return placement_value

def evaluate(self):
    white_points = 0
    black_points = 0
    for i in range(8):
        for j in range(8):
            if isinstance(self[i][j], ChessPiece):
                piece = self[i][j]
                if piece.color == 'white':
                    white_points += piece.get_score()
                    white_points += piece.x + piece.y
                    if piece.type == 'King':
                        white_points += self.calculate_king_safety(piece)
                    white_points += len(piece.filter_moves(piece.get_moves(self), self))*0.5
                else:
                    black_points += piece.get_score()
                    black_points += piece.x + piece.y
                    if piece.type == 'King':
                        black_points += self.calculate_king_safety(piece)
                    black_points += len(piece.filter_moves(piece.get_moves(self), self))*0.5

    if self.game_mode == 0:
        return black_points - white_points
    return white_points - black_points

## Task 4 -- Add two improvements to the Minimax algorithm choosing from Progressive Deepening, Transposition Tables, Opening Books, Move Ordering, Aspiration Window etc.

In [5]:
def get_hash_key(self):
    hash_key = 0
    for row in range(8):
        for col in range(8):
            piece = self.board[row][col]
            # verify if element from position i,j is a chess figure
            if isinstance(piece, ChessPiece):
                piece_type = type(piece).__name__
                color = piece.color
                # calculate hashkey for every piece and adding to total value
                hash_key ^= self.zobrist_keys[(piece_type, color, row, col)]
    return hash_key

def minimax_with_transposition(board, depth, alpha, beta, max_player, save_move, data):
    # calculate hashcode for table
    key = board.get_hash_key()
    # verify if key already exist in table
    if key in transposition_table:
        # return this piece
        return transposition_table[key]

    if depth == 0 or board.is_terminal():
        data[1] = board.evaluate()
        return data

    if max_player:
        max_eval = -math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color != board.get_player_color():
                    piece = board[i][j]
                    moves = piece.filter_moves(piece.get_moves(board), board)
                    for move in moves:
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        evaluation = minimax_with_transposition(board, depth - 1, alpha, beta, False, False, data)[1]
                        if save_move:
                            if evaluation >= max_eval:
                                if evaluation > data[1]:
                                    data.clear()
                                    data[1] = evaluation
                                    data[0] = [piece, move, evaluation]
                                elif evaluation == data[1]:
                                    data[0].append([piece, move, evaluation])
                        board.unmake_move(piece)
                        max_eval = max(max_eval, evaluation)
                        alpha = max(alpha, evaluation)
                        if beta <= alpha:
                            break
        return data
    else:
        min_eval = math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color == board.get_player_color():
                    piece = board[i][j]
                    moves = piece.get_moves(board)
                    for move in moves:
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        evaluation = minimax_with_transposition(board, depth - 1, alpha, beta, True, False, data)[1]
                        board.unmake_move(piece)
                        min_eval = min(min_eval, evaluation)
                        beta = min(beta, evaluation)
                        if beta <= alpha:
                            break

    transposition_table[key] = data
    # add key to table
    return data

def minimax_progressive_deepening(board, max_depth, alpha, beta, max_player, save_move, data):
    best_move = None
    for depth in range(1, max_depth + 1):
        # iterate for every deepening level from 1 to n+1
        result = minimax_with_transposition(board, depth, alpha, beta, max_player, save_move, data)
        if best_move is None or result[1] > best_move[1]:
            best_move = result
    return best_move

## Conclusions:

During this work I studied Minimax algorithm and Alpha-Beta Prunning for completing an existing Chess Engine.

## Bibliography:

1. Develop Chess Engine using minimax: https://www.freecodecamp.org/news/simple-chess-ai-step-by-step-1d55a9266977/
2. Minimax algorithm in chess: https://www.idtech.com/blog/minimax-algorithm-in-chess-checkers-tic-tac-toe
3. Alpha-Beta Prunning: https://www.chess.com/blog/the_real_greco/engines-navigating-the-tree-ab-pruning-minimax
4. Transpositional Tables: https://www.chessprogramming.org/Transposition_Table
5. Progressive Deepening: https://www.chessprogramming.org/Iterative_Deepening